## Some helpful code is provided bellow. It can serve to load data and organice functions.

In [1]:
#!pip install yfinance
import yfinance as yf
precios = yf.Ticker('XLI').history(period="10y", interval='1wk')

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC

In [3]:
precios = precios.Close

In [4]:
precios.head(10)

Date
2014-04-07    42.180496
2014-04-14    43.713589
2014-04-21    43.514694
2014-04-28    43.771591
2014-05-05    43.929054
2014-05-12    43.995335
2014-05-19    44.376530
2014-05-26    44.790882
2014-06-02    45.835041
2014-06-09    45.163799
Name: Close, dtype: float64

In [5]:
retornos = np.log(precios).diff()[1:].dropna()
mat_retornos = retornos.values

In [9]:
from sklearn.preprocessing import OneHotEncoder
np.set_printoptions(suppress=True)

In [54]:
def preparar(per = 20, umbral = 2):        
        
        # ventana de tiempo - Columnas
        muestras = len(mat_retornos) - per

        variables = np.zeros((1,per))
        l = len(mat_retornos)

        for x in range(muestras):
            variables = np.concatenate((variables, mat_retornos[l-per-x: l-x].reshape((1,per))))

        variables = variables[1:]

        X = variables[:,:-1]*100
        Y = variables[:,-1]*100
        
        neutral = [not x for x in ((Y>umbral)|(Y<-umbral))]
        
        Y[Y>umbral] = 2
        Y[Y<-umbral] = 0
        Y[neutral] = 1
        
        Y = OneHotEncoder(sparse=False).fit_transform(Y.reshape(-1,1))
        #print(Y[5,:])
        
        i_entrenamiento = int(len(variables)*0.7) #la fila hasta donde va el entrenamiento y donde comienza el conjunto de prueba
        
        x_train = X[:i_entrenamiento, :]
        x_test =  X[i_entrenamiento:, :]
        y_train = Y[:i_entrenamiento]
        y_test =  Y[i_entrenamiento:]
        
        return x_train, x_test, y_train, y_test

In [58]:
x_train, x_test, y_train, y_test = preparar(per = 20, umbral = 1)

In [56]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense


In [61]:
# Crear el modelo
model = Sequential()

capas = [5,3]

# Agregar capa oculta
model.add(Dense(capas[0], input_dim=x_train.shape[1], activation='sigmoid'))
model.add(Dense(capas[1], input_dim=capas[0], activation='sigmoid'))
#model.add(Dense(3, input_dim=3, activation='tanh'))

In [62]:
# Agregar capa de salida
model.add(Dense(3, activation='sigmoid'))

In [63]:
# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(x_train, y_train, epochs=300, verbose=1)

Epoch 1/300
11/11 [==============================] - 1s 4ms/step - loss: 0.2378 - accuracy: 0.2793
Epoch 2/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2364 - accuracy: 0.2763
Epoch 3/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2351 - accuracy: 0.2703
Epoch 4/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2339 - accuracy: 0.2703
Epoch 5/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2328 - accuracy: 0.2853
Epoch 6/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2317 - accuracy: 0.2793
Epoch 7/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2307 - accuracy: 0.3003
Epoch 8/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2297 - accuracy: 0.2913
Epoch 9/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2288 - accuracy: 0.3033
Epoch 10/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2280 - accuracy: 0.3183
Epoch 11/

In [64]:
# Evaluar el modelo
loss, accuracy = model.evaluate(x_train, y_train)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Predicción

11/11 [==============================] - 0s 2ms/step - loss: 0.1919 - accuracy: 0.5766
Loss: 0.1918998658657074, Accuracy: 0.5765765905380249
5/5 [==============================] - 0s 5ms/step
Predicción final:
[0.57114226 0.50012046 0.5162743  0.5311422  0.6022969  0.55017465
 0.50578874 0.5892636  0.67357624 0.5058648  0.54184544 0.66810864
 0.5735344  0.5684922  0.5649032  0.507909   0.50965154 0.51057
 0.57576096 0.6100852  0.51008534 0.52958685 0.5420181  0.53895456
 0.55085206 0.72274554 0.57674706 0.6544927  0.5715285  0.6833667
 0.56722724 0.71544856 0.6576423  0.6984029  0.5365919  0.66935235
 0.5507248  0.649928   0.62208074 0.50644743 0.6060325  0.64308
 0.50225484 0.5913062  0.55533594 0.50800467 0.5127758  0.6794161
 0.52106845 0.70329666 0.5186444  0.5089973  0.53945374 0.54944366
 0.6545456  0.568243  ]


In [74]:
model.predict(x_train)

11/11 [==============================] - 0s 2ms/step


array([[0.26233405, 0.4949322 , 0.24026917],
       [0.2538181 , 0.5322305 , 0.21880971],
       [0.24578457, 0.5247973 , 0.23137477],
       [0.35473496, 0.3172512 , 0.31421676],
       [0.23530468, 0.40543026, 0.338673  ],
       [0.3234763 , 0.40662292, 0.25145343],
       [0.24477331, 0.49905732, 0.2515346 ],
       [0.22671783, 0.21785368, 0.57683045],
       [0.28365672, 0.46441862, 0.24166968],
       [0.2325883 , 0.16175103, 0.6668088 ],
       [0.34419543, 0.4426557 , 0.23909701],
       [0.24781989, 0.47783756, 0.2683112 ],
       [0.24340066, 0.5240025 , 0.23442261],
       [0.23322074, 0.41979074, 0.33708254],
       [0.34285972, 0.4758432 , 0.21466418],
       [0.22567624, 0.4754726 , 0.289575  ],
       [0.27618578, 0.52961123, 0.20157604],
       [0.28747144, 0.47509465, 0.2197802 ],
       [0.23624837, 0.5382514 , 0.23191318],
       [0.28318822, 0.2450733 , 0.47447556],
       [0.32206753, 0.48276305, 0.22758032],
       [0.27026606, 0.2222095 , 0.53149   ],
       [0.